from
https://mne.tools/stable/auto_tutorials/preprocessing/10_preprocessing_overview.html#

In [1]:
# Authors: The MNE-Python contributors.
# License: BSD-3-Clause
# Copyright the MNE-Python contributors.
import os

import numpy as np

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

Using default location ~/mne_data for sample...
Creating C:\Users\mdr24\mne_data


100%|#####################################| 1.65G/1.65G [00:00<00:00, 3.40TB/s]
Untarring contents of 'C:\Users\mdr24\mne_data\MNE-sample-data-processed.tar.gz' to 'C:\Users\mdr24\mne_data'


Attempting to create new mne-python configuration file:
C:\Users\mdr24\.mne\mne-python.json
Download complete in 01m01s (1576.2 MB)
Opening raw data file C:\Users\mdr24\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 36037  =      0.000 ...    60.000 secs...


<Raw | sample_audvis_raw.fif, 376 x 36038 (60.0 s), ~106.6 MiB, data loaded>

What are artifacts?
Artifacts are parts of the recorded signal that arise from sources other than the source of interest (i.e., neuronal activity in the brain). As such, artifacts are a form of interference or noise relative to the signal of interest. There are many possible causes of such interference, for example:

Environmental artifacts
Persistent oscillations centered around the AC power line frequency (typically 50 or 60 Hz)

Brief signal jumps due to building vibration (such as a door slamming)

Electromagnetic field noise from nearby elevators, cell phones, the geomagnetic field, etc.

Instrumentation artifacts
Electromagnetic interference from stimulus presentation (such as EEG sensors picking up the field generated by unshielded headphones)

Continuous oscillations at specific frequencies used by head position indicator (HPI) coils

Random high-amplitude fluctuations (or alternatively, constant zero signal) in a single channel due to sensor malfunction (e.g., in surface electrodes, poor scalp contact)

Biological artifacts
Periodic QRS-like signal patterns (especially in magnetometer channels) due to electrical activity of the heart

Short step-like deflections (especially in frontal EEG channels) due to eye movements

Large transient deflections (especially in frontal EEG channels) due to blinking

Brief bursts of high frequency fluctuations across several channels due to the muscular activity during swallowing

There are also some cases where signals from within the brain can be considered artifactual. For example, if a researcher is primarily interested in the sensory response to a stimulus, but the experimental paradigm involves a behavioral response (such as button press), the neural activity associated with the planning and executing the button press could be considered an artifact relative to signal of interest (i.e., the evoked sensory response).

Note

Artifacts of the same genesis may appear different in recordings made by different EEG or MEG systems, due to differences in sensor design (e.g., passive vs. active EEG electrodes; axial vs. planar gradiometers, etc).
What to do about artifacts
There are 3 basic options when faced with artifacts in your recordings:

Ignore the artifact and carry on with analysis

Exclude the corrupted portion of the data and analyze the remaining data

Repair the artifact by suppressing artifactual part of the recording while (hopefully) leaving the signal of interest intact

There are many different approaches to repairing artifacts, and MNE-Python includes a variety of tools for artifact repair, including digital filtering, independent components analysis (ICA), Maxwell filtering / signal-space separation (SSS), and signal-space projection (SSP). Separate tutorials demonstrate each of these techniques for artifact repair. Many of the artifact repair techniques work on both continuous (raw) data and on data that has already been epoched (though not necessarily equally well); some can be applied to memory-mapped data while others require the data to be copied into RAM. Of course, before you can choose any of these strategies you must first detect the artifacts, which is the topic of the next section.

Artifact detection
MNE-Python includes a few tools for automated detection of certain artifacts (such as heartbeats and blinks), but of course you can always visually inspect your data to identify and annotate artifacts as well.

We saw in the introductory tutorial that the example data includes SSP projectors, so before we look at artifacts let’s set aside the projectors in a separate variable and then remove them from the Raw object using the del_proj() method, so that we can inspect our data in it’s original, raw state:





In [2]:
ssp_projectors = raw.info["projs"]
raw.del_proj()

<Raw | sample_audvis_raw.fif, 376 x 36038 (60.0 s), ~106.6 MiB, data loaded>